In [1]:
import chromadb

from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama    
from langchain.prompts import ChatPromptTemplate

In [2]:
db = Chroma(
        persist_directory="chroma",
        embedding_function=HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-small",
            model_kwargs={'device': 'cpu'}
        )
    )

retriever = db.as_retriever()

c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\50272\anaconda3\envs\law_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated vers

In [7]:
from langchain.chains import RetrievalQA


model_name = "aya"
llm = Ollama(model=model_name)
chain = RetrievalQA.from_llm(llm=llm, 
                    retriever=retriever
                    )

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = Ollama(model="aya")

system_prompt = (
    "Gunakan konteks yang diberikan untuk menjawab pertanyaan. "
    "Kalau anda tidak tahu jawabannya, cukup jawab dengan Maaf, saya tidak tahu. "
    "Jawaban harus terdiri dari minimal tiga paragraf. "
    "Konteks: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

query = "Apa tugas dan wewenang MK?"

chain.invoke({"input": query})

In [14]:
PROMPT_TEMPLATE = """
Anda adalah seorang paralegal hukum AI. Gunakan konteks berikut
untuk menjawab pertanyaan yang diberikan.

{context}

---

Pertanyaan: {question}
Jawaban yang anda berikan harus jelas dan minimal terdiri dari 2 paragraf.
Jika anda tidak tahu jawabannya, cukup jawab dengan "Maaf, saya tidak tahu".
"""

In [21]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. Provide the answer in Indonesian Language."
    "\n\n"
    "{context}"
)

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [28]:
response = rag_chain.invoke({"input": "Siapa kiper timnas Prancis saat ini?"})

print(response)

{'input': 'Siapa kiper timnas Prancis saat ini?', 'context': [Document(metadata={'page': 0, 'source': '/content/PP/2020/Peraturan Pemerintah Nomor 41 Tahun 20200_fix.pdf'}, page_content='LEMBARAN NEGARA \nREPUBLIK INDONESIA \nNo. 181, 2020  ADMINISTRASI. Pengalihan Pegawai KPK . Pegawai \nASN.  \n \nPERATURAN PEMERINTAH REPUBLIK INDONESIA  \nNOMOR 41 TAHUN 2020 \nTENTANG \nPENGALIHAN PEGAWAI KOMISI PEMBERANTASAN TINDAK PIDANA \nKORUPSI MENJADI PEGAWAI APARATUR SIPIL NEGARA \n  \nDENGAN RAHMAT TUHAN YANG MAHA ESA  \n \nPRESIDEN REPUBLIK INDONESIA, \n \n \nMenimbang : a. bahwa Komisi Pemberantasan Tindak Pidana Korupsi \nsebagai lembaga negara dalam rumpun kekuasaan \neksekutif yang melaksanakan tugas pencegahan dan \npemberantasan Tindak Pidana Korupsi, dalam \npelaksanaan tugasnya perlu mendapatkan dukungan \ndari Aparatur Sipil Negara; \nb. bahwa untuk menindaklanjuti Pasal 1 angka 6, Pasal \n69B, dan Pasal 69C Undang-Undang Nomor 30 Tahun \n2002 tentang Komisi Pemberantasan Tindak Pi

In [15]:

def get_llm_response(query):    
    # result = retriever.invoke(input=query)
    result = db.similarity_search(query, k=5)

    context = "\n\n---\n\n".join([doc.page_content for doc in result])
    
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = template.format(context=context, question=query)
    
    answer = chain.invoke({
        "input_documents": result,
        "query": prompt
    })
    
    response = {
        "response": answer['result'],
    }

    return answer

In [17]:
query = "Apa inti dari pembukaan UUD 1945 Bait ke dua?"

get_llm_response(query)

{'input_documents': [Document(metadata={'page': 0, 'source': '/content/UUD/1945/Undang-Undang Dasar Republik Indonesia Tahun 1945_fix.pdf'}, page_content='UNDANG -UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\nBahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tida k sesuai dengan peri -\nkemanusiaan dan peri -keadilan.\nDan perjuangan pergerakan kemerdekaan Indonesia telah sam pailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan raky at Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\nAtas berkat rakhmat Allah Yang Maha Kuasa dan dengan didoron gkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rak yat Indonesia menyatakan\ndengan ini kemerdekaannya.\nKemudian dari pada itu untuk membentuk suatu Pemerintah Neg ara Indonesia yang\nmelindungi segenap bang

In [20]:
query2 = "Apa saja tugas DPR?"

get_llm_response(query2)

{'input_documents': [Document(metadata={'page': 4, 'source': '/content/UUD/1945/Undang-Undang Dasar Republik Indonesia Tahun 1945_fix.pdf'}, page_content='BAB VI\nPEMERINTAH DAERAH\nPasal 18\nPembagian daerah Indonesia atas daerah be sar dan kecil, dengan bentuk susunan\npemerintahannya ditetapkan dengan undang -undang, dengan memandang dan\nmengingati dasar permusyawaratan dalam sistem pemerintah an negara, dan hak -hak\nasal-usul dalam daerah -daerah yang bersifat istimewa.\nBAB VII\nDEWAN PE RWAKILAN RAKYAT\nPasal 19\n(1)Susunan Dewan Perwakilan Rakyat ditetapkan dengan undang -undang.\n(2)Dewan Perwakilan Rakyat bersidang sedikitnya sekali dalam setahun.\nPasal 20\n(1)Tiap-tiap undang -undang menghendaki persetujuan Dewan Perwakilan Rakyat.\n(2)Jika sesuatu rancangan undang -undang tidak mendapat persetujuan Dewan\nPerwakilan Rakyat, maka rancangan tadi tidak boleh dimajuk an lagi dalam\npersidangan Dewan Perwakilan Rakyat masa itu.\nPasal 21\n(1)Anggota -anggota Dewan Perwakilan R

In [16]:
query3 = "Siapa kiper terbaik timnas Indonesia sepanjang masa?"

get_llm_response(query3)

{'input_documents': [Document(metadata={'page': 77, 'source': '/content/PP/2021/Peraturan Pemerintah Nomor 114 Tahun 20210_fix.pdf'}, page_content='PRESIDEN\nREPIJBLIK INDONESIA\nC-D\nHimne dan Mars UNP\n1. Himne UNP sebagai berikut:\nSK No 101198 A2. Mars'),
  Document(metadata={'page': 11, 'source': '/content/UU/2015/Undang-Undang Nomor 12 Tahun 20150_fix.pdf'}, page_content='No.5741-12-\n45.BadanKoordinasiPenanamanModal WTP WTP\n46.BadanNarkotikaNasional WTP WTP\n47.KementerianPembangunanDaerah\nTertinggalWDP WTP\n48.Badan Kependudukan dan Keluarga\nBerencanaNasionalWDP WTP\n49.KomisiNasionalHakAsasiManusia WTP WTP\n50.BadanMeteorologi,Klimatologi,dan\nGeofisikaWDP WTP\n51.KomisiPemilihanUmum WDP WDP\n52.MahkamahKonstitusi WTP WTP\n53.PusatPelaporandanAnalisis\nTransaksiKeuanganWTP WTP\n54.LembagaIlmuPengetahuanIndonesia WTP WTP\n55.BadanTenagaNuklirNasional WTP WTP\n56.BadanPengkaj iandanPenerapan\nTeknologiWDP WTP\n57.LembagaPenerbangandanAntariksa\nNasionalWDP WDP\n58.BadanInform